# [초급 프로젝트] 4팀_김명환 - YOLO모델링 테스트 및 평가

---
---

# 환경설정
    - 라이브러리 설치 및 로딩
    - 사용자 함수 사용

In [1]:
!pip install -q gdown
!pip install -q albumentations
!pip install -q ultralytics
!pip install -q -U ultralytics
!pip install -q nbformat
!pip install -q roboflow
!pip install -q opencv-python
!pip install -q opencv-python-headless
!pip install -q wandb
print("로딩완료")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 71.7 MB/s eta 0:00:00
로딩완료


In [2]:
# WandB(Weights & Biases) 로그인 명령어 - API 키를 사용하여 인증
!wandb login 86a7b8c07184b2efdfb116546a17b1905e41cb5d

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [3]:
# 기본 라이브러리

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import average_precision_score

# --- 기타 라이브러리 ---
import cv2
from PIL import Image
from PIL import ImageFilter
from PIL import ImageDraw
import albumentations as A
import IPython.display
#from tqdm import tqdm
from tqdm.notebook import tqdm

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from collections import OrderedDict
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 기타 ---
import re
import os
import csv  # 추가 필요
import sys
import copy
import json
import math
import random
import yaml
import shutil
import requests
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from datetime import datetime
from datetime import timezone, timedelta
from pathlib import Path
import pytz
__kst = pytz.timezone('Asia/Seoul')

# GPU 설정
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

  # 재현 가능한 결과를 위해
np.random.seed(42)
torch.manual_seed(42)
if __device == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치:{__device}")

라이브러리 로드 완료 사용장치:cuda


In [4]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.0.2 (v2.x+): 호환성 모드 적용됨
install fonts-nanum...
Mounted at /content/drive
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = /content/drive/MyDrive
🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.0.2 (v2.x+): 호환성 모드 적용됨
Mounted at /content/drive
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = /content/drive/MyDrive


<module 'helper_c0z0c_dev' from '/content/helper_c0z0c_dev.py'>

In [5]:
# OS 및 경로 관련 라이브러리 임포트
import os, sys
from pathlib import Path

# 유틸리티 함수 디렉토리 경로 설정 (Colab 환경 여부에 따라 다르게 지정)
utils_dir = None
if helper.is_colab:
    # Colab 환경일 경우 Google Drive 경로 사용
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    # 로컬 환경일 경우 현재 드라이브 기준 경로 사용
    utils_dir = os.path.join(Path.cwd().drive + '\\\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

# 유틸리티 함수 경로를 파이썬 모듈 검색 경로에 추가
sys.path.append(str(utils_dir))
print("sys.path:", sys.path)

# health_ea_utils 모듈 임포트 및 최신화(reload)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

# helper 및 health_ea_utils 파일 경로 출력 (디버깅용)
print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)

utils_dir: /content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils
sys.path: ['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpttgrx2np', '/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils']
🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.0.2 (v2.x+): 호환성 모드 적용됨
Mounted at /content/drive
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = /content/drive/MyDrive
🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.0.2 (v2.x+): 호환성 모드 적용됨
Mounted at /content/drive
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = /content/drive/MyDrive
helper.__file__: /content/helper_c0z0c_dev.py
health_ea_utils.__file__: /content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils/health_ea_utils.py


In [6]:
def get_tqdm_kwargs():
    """Widget 오류를 방지하는 안전한 tqdm 설정"""
    return {
        'disable': False,
        'leave': True,
        'file': sys.stdout,
        'ascii': True,  # ASCII 문자만 사용
        'dynamic_ncols': False,
#        'ncols': 80  # 고정 폭
    }

def drive_root():
    """
    Google Drive의 최상위 경로를 반환하는 함수입니다.
    - 로컬 환경(Windows): D:\GoogleDrive
    - Colab 환경: /content/drive/MyDrive
    프로젝트 내에서 데이터, 모델, 설정 파일 등 경로를 일관되게 관리할 때 사용합니다.
    """
    root_path = os.path.join(Path.cwd().drive + '\\', "GoogleDrive")
    if helper.is_colab:
        root_path = os.path.join("/content/drive/MyDrive")
    return root_path

def get_path_modeling(add_path = None):
    """
    get_path_modeling() 함수는 모델링 관련 파일(예: 학습 결과, 체크포인트, 로그 등)을 저장할 경로를 반환합니다.
    기본적으로 Google Drive의 루트 경로(drive_root()) 아래 "modeling_yolo" 폴더를 기준으로 경로를 생성합니다.
    추가 하위 경로가 필요할 경우 add_path 인자를 통해 세부 폴더까지 지정할 수 있습니다.
    예시:
    get_path_modeling() → modeling_yolo (로컬)
    get_path_modeling("exp1") → D:\GoogleDrive\modeling_yolo\exp1
    Colab 환경에서는 /content/drive/MyDrive/modeling_yolo로 반환됩니다.
    """

    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def get_path_modeling_release(add_path = None):
    """
    get_path_modeling_release() 함수는 모델링 결과물(예: 학습 결과, 체크포인트, 로그 등)을 저장할 경로를 반환합니다.
    기본적으로 Google Drive의 루트 경로(drive_root()) 아래 "modeling_yolo" 폴더를 기준으로 경로를 생성합니다.
    추가 하위 경로가 필요할 경우 add_path 인자를 통해 세부 폴더까지 지정할 수 있습니다.
    예시:
    get_path_modeling_release() → modeling_yolo (로컬)
    get_path_modeling_release("exp1") → D:\GoogleDrive\modeling_yolo\exp1
    Colab 환경에서는 /content/drive/MyDrive/modeling_yolo로 반환됩니다._
    """
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def print_dir_tree(root, max_depth=2, list_count=3, indent=""):
    """
    지정한 폴더(root) 하위의 디렉토리 구조를 트리 형태로 출력하는 함수입니다.

    Args:
        root: 시작 경로(폴더)
        max_depth: 출력할 최대 깊이(디폴트 2)
        list_count: 파일 개수가 많을 때 몇 개만 출력할지(디폴트 3)
        indent: 들여쓰기(재귀적으로 사용)
    """
    import os
    if max_depth < 0:
        return
    try:
        items = os.listdir(root)
    except Exception as e:
        print(indent + f"[Error] {e}")
        return

    img_count = len([f for f in os.listdir(root) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
    for item in items:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            print(indent + "|-- "+ item)
            # 이미지 파일 개수만 출력
            img_count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
            if img_count > list_count:
                print(indent + "   "+ f"[데이터파일: {img_count}개]")
            print_dir_tree(root=path, max_depth=max_depth-1, list_count=list_count, indent=indent + "   ")
        else:
            if list_count < img_count and item.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt')):
                continue
            print(indent + "|-- "+ item)

def save_model_dict(model, path, pth_name, kwargs=None):
    """모델 state_dict와 추가 정보를 저장"""
    def safe_makedirs(path):
        """안전한 디렉토리 생성"""
        if os.path.exists(path) and not os.path.isdir(path):
            os.remove(path)  # 파일이면 삭제
        os.makedirs(path, exist_ok=True)

    # 디렉토리 생성
    safe_makedirs(path)

    # 모델 구조 정보 추출
    model_info = {
        'class_name': model.__class__.__name__,
        'init_args': {},
        'str': str(model),
        'repr': repr(model),
        'modules': [m.__class__.__name__ for m in model.modules()],
    }

    # 생성자 인자 자동 추출(가능한 경우)
    if hasattr(model, '__dict__'):
        for key in ['in_ch', 'base_ch', 'num_classes', 'out_ch']:
            if hasattr(model, key):
                model_info['init_args'][key] = getattr(model, key)

    # kwargs 처리
    extra_info = {}
    if kwargs is not None:
        if isinstance(kwargs, str):
            extra_info = json.loads(kwargs)
        elif isinstance(kwargs, dict):
            extra_info = kwargs

    model_info.update(extra_info)

    # 저장할 dict 구성
    save_dict = {
        'model_state': model.state_dict(),
        'class_name': model.__class__.__name__,
        'model_info': model_info,
    }

    save_path = os.path.join(path, f"{pth_name}.pth")
    torch.save(save_dict, save_path)
    return save_path

def load_model_dict(path, pth_name=None):
    """
    save_model_dict로 저장한 모델을 불러오는 함수
    반환값: (model_state, model_info)
    """
    import torch
    load_path = path
    if pth_name is not None:
        load_path = os.path.join(path, f"{pth_name}.pth")
    checkpoint = torch.load(load_path, map_location='cpu', weights_only=False)  # <-- 여기 추가
    model_state = checkpoint.get('model_state')
    model_info = checkpoint.get('model_info')
    model_info['file_name'] = os.path.basename(load_path)
    return model_state, model_info


def search_pth_files(base_path):
    """
    입력된 경로의 하위 폴더들에서 pth 파일들을 검색
    """
    pth_files = []

    if not os.path.exists(base_path):
        print(f"경로가 존재하지 않습니다: {base_path}")
        return pth_files

    print(f"pth 파일 검색 시작: {base_path}")

    # 하위 폴더들을 순회하며 pth 파일 검색
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pth'):
                pth_path = os.path.join(root, file)
                pth_files.append(pth_path)

    # 결과 정리 및 출력
    if pth_files:
        print(f"\n발견된 pth 파일들 ({len(pth_files)}개):")
        for i, pth_file in enumerate(pth_files, 1):
            # 상대 경로로 표시 (base_path 기준)
            rel_path = os.path.relpath(pth_file, base_path)
            print(f" {i:2d}. {rel_path}")
    else:
        print("pth 파일을 찾을 수 없습니다.")

    return pth_files

def print_json_tree(data, indent="", max_depth=4, _depth=0, list_count=2, print_value=True):
    """
    JSON 객체를 지정한 단계(max_depth)까지 트리 형태로 출력
    - list 타입은 3개 이상일 때 개수만 출력
    - 하위 노드가 값일 경우 key(type) 형태로 출력
    - print_value=True일 때 key(type): 값 형태로 출력
    """
    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}|-- {key}")
                print_json_tree(value, indent + "    ", max_depth, _depth + 1, list_count, print_value)
            else:
                if print_value:
                    print(f"{indent}|-- {key}({type(value).__name__}): {value if len(str(value)) < 100 else f'{str(value)[:30]}...'}")
                else:
                    print(f"{indent}|-- {key}({type(value).__name__})")
    elif isinstance(data, list):
        if len(data) > list_count:
            print(f"{indent}|-- [list] ({len(data)} items)")
        else:
            for i, item in enumerate(data):
                if isinstance(item, (dict, list)):
                    print(f"{indent}|-- [{i}]")
                    print_json_tree(item, indent + "    ", max_depth, _depth + 1, list_count, print_value)
                else:
                    if print_value:
                        print(f"{indent}|-- [{i}]({type(item).__name__}): {item if len(str(item)) < 100 else f'{str(item)[:30]}...'}")
                    else:
                        print(f"{indent}|-- [{i}]({type(item).__name__})")
    else:
        if print_value:
            print(f"{indent}{type(data).__name__}: {data if len(str(data)) < 100 else f'{str(data)[:30]}...'}")
        else:
            print(f"{indent}{type(data).__name__}")

def print_git_tree(data, indent="", max_depth=3, _depth=0):
    """
    PyTorch tensor/딕셔너리/리스트를 git tree 스타일로 출력
    """
    import torch
    import numpy as np

    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}├─ {key} [{type(value).__name__}]")
            print_git_tree(value, indent + "│  ", max_depth, _depth + 1)
    elif isinstance(data, (list, tuple)):
        for i, item in enumerate(data):
            print(f"{indent}├─ [{i}] [{type(item).__name__}]")
            print_git_tree(item, indent + "│  ", max_depth, _depth + 1)
    elif torch.is_tensor(data):
        shape = tuple(data.shape)
        dtype = str(data.dtype)
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ Tensor shape={shape} dtype={dtype} preview={preview_str}")
    elif isinstance(data, np.ndarray):
        shape = data.shape
        dtype = data.dtype
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ ndarray shape={shape} dtype={dtype} preview={preview_str}")
    else:
        val_str = str(data)
        print(f"{indent}└─ {type(data).__name__}: {val_str[:80]}{'...' if len(val_str)>80 else ''}")


print("유틸리티 함수 로드 완료")

유틸리티 함수 로드 완료


# 프로그래밍

## 데이타 다운로드

In [7]:
# download_files 변수 예시 (Google Drive 및 Naver MyBox 링크)
# 여러 버전의 데이터셋 다운로드 링크를 주석으로 관리

# download_files={
#     'yolo_label_one_class' : r'https://drive.google.com/file/d/177_86k4BuT6JnFnq7ZHJtEjp7jaRbCl2/view?usp=sharing',
#     'yolo_label' : r'https://drive.google.com/file/d/1nc-WFcw7lCS7s7VGzN9Kxh80PiBBggez/view?usp=sharing',
#     'yolo_resize_one_class' : r'https://drive.google.com/file/d/1Ak0EvkMnuwvcAFvTO-zovIgVcNlROjsS/view?usp=sharing',
#     'yolo_resize' : r'https://drive.google.com/file/d/1kpo57qOJhEhrkuzUCEh57ILB5xSPVoFv/view?usp=sharing',
# }

# download_files={
#     'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
#     'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
#     'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
#     'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
# }

# 실제로 사용할 데이터셋 다운로드 링크만 활성화
download_files={
    # 'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
    # 'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
    # 'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
    # 'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
    'yolo_noresize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107',
    # 'yolo_noresize_one_class':r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDgwMDcxODk0NHxGfDA&svcType=MYBOX-WEB&time=1757893856220',
}

# 참고용: 각 데이터셋의 직접 다운로드 링크
# yolo_noresize = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107
# yolo_noresize_one_class = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDgwMDcxODk0NHxGfDA&svcType=MYBOX-WEB&time=1757893856220


In [8]:
import gdown
def download_gdrive_file(url, output_path, ignore=True):
    """Google Drive 파일 다운로드 함수

    Args:
        url (str): Google Drive 공유 링크
        output_path (str): 다운로드할 파일 경로
        ignore (bool, optional): True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀. Defaults to True.

    Raises:
        ValueError: Google Drive 파일 ID를 찾을 수 없습니다.
    """
    # 공유 링크에서 파일 ID 추출
    if os.path.exists(output_path):
        if ignore:
            os.remove(output_path)
        else:
            return

    file_id_match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if not file_id_match:
        raise ValueError("Google Drive 파일 ID를 찾을 수 없습니다.")
    file_id = file_id_match.group(1)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

def download_http(url, target, ignore=True):
    """
    HTTP 파일 다운로드 함수 (진행률 표시)
    url: 다운로드할 파일 URL
    target: 저장할 파일 경로
    ignore: True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀
    """
    if os.path.exists(target):
        if ignore:
            os.remove(target)
        else:
            print(f"이미 파일이 존재합니다: {target}")
            return target

    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(target, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(target)}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        ascii=True
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    print(f"다운로드 완료: {target}")
    return target

# local_code_it_ai04 = os.path.join( '~/.cache/' if helper.is_colab else Path.cwd().drive + '\\'
#                                   ,'temp'
#                                   , 'code_it_ai04')

# 데이터 다운로드 및 압축 해제 코드에 상세 주석 추가

# 다운로드 경로 설정 (Colab/로컬 환경에 따라 다름)
if helper.is_colab:
    local_code_it_ai04 = os.path.join( '/content/', 'code_it_ai04')
else:
    local_code_it_ai04 = os.path.join( Path.cwd().drive + '\\\\', 'temp', 'code_it_ai04')

print("local_code_it_ai04:", local_code_it_ai04)

os.makedirs(local_code_it_ai04, exist_ok=True)  # 폴더 생성 코드 추가

unzip_paths = []
for key, url in download_files.items():
    print(f"{key}: {url}")
    zipfile = os.path.join(local_code_it_ai04, f'{key}.zip')
    unzip_path = os.path.join(local_code_it_ai04, f'{key}.zip.unzip')
    # 이미 압축해제된 폴더가 있으면 재다운로드/압축해제하지 않음
    if os.path.exists(unzip_path):
        print(f"이미 압축해제된 폴더가 존재합니다: {unzip_path}")
        print('unzipfile:', unzip_path)
        unzip_paths.append(unzip_path)
        continue
    # Google Drive 파일 다운로드 함수 (주석처리, 필요시 사용)
    #download_gdrive_file(url, os.path.join(local_code_it_ai04, f'{key}.zip'), ignore=False)
    # 일반 HTTP 다운로드 함수 사용
    download_http(url, zipfile, ignore=False)
    # 압축 해제 (health_ea_utils의 unzip 함수 사용)
    unzip_path_list = heu.unzip([os.path.join(local_code_it_ai04, f'{key}.zip')])
    # 압축 해제된 경로 리스트 출력 및 저장
    print('unzip_path_list:', unzip_path_list)
    unzip_paths.extend(unzip_path_list)


local_code_it_ai04: /content/code_it_ai04
yolo_noresize: https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107


다운로드 완료: /content/code_it_ai04/yolo_noresize.zip


압축 해제 중: yolo_noresize.zip: 100%|██████████| 3822/3822 [00:02<00:00, 1781.28file/s]

압축 해제 완료: /content/code_it_ai04/yolo_noresize.zip.unzip
unzip_path_list: ['/content/code_it_ai04/yolo_noresize.zip.unzip']


In [9]:
# google drive root에 keggle.json 파일 필요합니다.
for path in unzip_paths:
    print("압축해제된 폴더:", path)

yolo_dataset_path = os.path.join(local_code_it_ai04, f'yolo_label_one_class.zip.unzip')
yaml_path = os.path.join(yolo_dataset_path, "dataset.yaml")

def get_path_data():
    path = yolo_dataset_path
    return path

print("yaml_path:", yaml_path)
print("get_path_data:", get_path_data())

압축해제된 폴더: /content/code_it_ai04/yolo_noresize.zip.unzip
yaml_path: /content/code_it_ai04/yolo_label_one_class.zip.unzip/dataset.yaml
get_path_data: /content/code_it_ai04/yolo_label_one_class.zip.unzip


## YOLO 모델링

In [10]:
from pathlib import Path
from ultralytics import YOLO

# GPU 확인
device = __device
print(f"사용 디바이스: {device}")
print(f"CUDA 버전: {torch.version.cuda}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
사용 디바이스: cuda
CUDA 버전: 12.6


In [11]:
for path in unzip_paths:
    print(f"압축해제된 폴더: {path}")


압축해제된 폴더: /content/code_it_ai04/yolo_noresize.zip.unzip


In [12]:
def extract_folder_key(folder_path):
    """
    폴더 경로에서 yolo_label_one_class 등 주요 명칭만 추출
    예시: 'd:\\temp\\code_it_ai04\\yolo_label_one_class.zip.unzip' → 'yolo_label_one_class'
    """
    # 경로에서 마지막 폴더/파일명 추출
    base = os.path.basename(folder_path)
    # .zip 또는 .zip.unzip 등 확장자 제거
    key = re.sub(r'\.zip(\.unzip)?$', '', base)
    return key

def update_yaml_paths_to_absolute(yaml_path):
    """YAML 파일의 상대 경로를 절대 경로로 업데이트합니다.

    Args:
        yaml_path (str): 업데이트할 YAML 파일 경로
    """
    # dataset.yaml 파일을 읽어서 내부 경로(path)를 절대경로로 변환
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    yaml_dir = os.path.dirname(yaml_path)
    # 'path' 항목을 yaml 파일 위치 기준 절대경로로 변경
    data['path'] = os.path.normpath(os.path.join(yaml_dir, data['path']))
    # 아래 코드는 필요시 train/val/test 경로도 절대경로로 변경 가능
    # for key in ['train', 'val', 'test']:
    #     if key in data and not os.path.isabs(data[key]):
    #         data[key] = os.path.normpath(os.path.join(yaml_dir, data[key]))

    # 변경된 내용을 다시 yaml 파일에 저장
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, allow_unicode=True)

import wandb
wandb.login()  # 또는 wandb.init(project="your_project_name")
# 한글 폰트 및 마이너스 기호 깨짐 방지 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False


wandb: Currently logged in as: c0z0c-dev (c0z0c-dev-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 모델 학습

In [13]:
def log_training_metrics(results, model_name):
    """
    YOLO 훈련 결과에서 메트릭을 추출하여 WandB에 로그합니다.
    Args:
        results: YOLO 모델 훈련 결과 객체
        model_name: 모델 이름 (str)
    """
    try:
        # results.csv 파일에서 메트릭 읽기
        csv_path = os.path.join(results.save_dir, 'results.csv')
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            df.columns = df.columns.str.strip()  # 공백 제거

            # 각 에포크별 메트릭 로그
            for idx, row in df.iterrows():
                epoch = idx + 1

                # 메트릭 딕셔너리 생성
                metrics = {
                    "epoch": epoch,
                    "train/box_loss": row.get('train/box_loss', 0),
                    "train/cls_loss": row.get('train/cls_loss', 0),
                    "train/dfl_loss": row.get('train/dfl_loss', 0),
                    "val/box_loss": row.get('val/box_loss', 0),
                    "val/cls_loss": row.get('val/cls_loss', 0),
                    "val/dfl_loss": row.get('val/dfl_loss', 0),
                    "metrics/precision(B)": row.get('metrics/precision(B)', 0),
                    "metrics/recall(B)": row.get('metrics/recall(B)', 0),
                    "metrics/mAP50(B)": row.get('metrics/mAP50(B)', 0),
                    "metrics/mAP50-95(B)": row.get('metrics/mAP50-95(B)', 0),
                }

                # NaN 값 처리
                metrics = {k: (v if not pd.isna(v) else 0) for k, v in metrics.items()}

                wandb.log(metrics)

            print(f"훈련 메트릭이 WandB에 로그되었습니다: {len(df)}개 에포크")

        # 추가 시각화: 혼동 행렬, PR 커브 등
        log_additional_plots(results)

    except Exception as e:
        print(f"메트릭 로깅 중 오류: {e}")

def log_additional_plots(results):
    """
    추가 플롯들을 WandB에 로그
    """
    try:
        save_dir = results.save_dir

        # 1. 훈련 결과 플롯
        plots_to_log = [
            ('results.png', 'Training Results'),
            ('confusion_matrix.png', 'Confusion Matrix'),
            ('PR_curve.png', 'Precision-Recall Curve'),
            ('F1_curve.png', 'F1 Curve'),
        ]

        for plot_file, title in plots_to_log:
            plot_path = os.path.join(save_dir, plot_file)
            if os.path.exists(plot_path):
                wandb.log({f"plots/{title}": wandb.Image(plot_path)})

        # 2. 검증 배치 이미지
        val_batch_path = os.path.join(save_dir, 'val_batch0_pred.jpg')
        if os.path.exists(val_batch_path):
            wandb.log({"validation/predictions": wandb.Image(val_batch_path)})

        print("추가 플롯들이 WandB에 로그되었습니다.")

    except Exception as e:
        print(f"추가 플롯 로깅 중 오류: {e}")

def log_comprehensive_metrics(results, model, yaml_path):
    """
    포괄적인 메트릭 및 시각화 로깅
    """
    try:
        # 1. 기본 훈련 메트릭
        log_training_metrics(results, "comprehensive")

        # 2. 모델 요약 정보
        model_summary = {
            "model/parameters": sum(p.numel() for p in model.model.parameters()),
            "model/layers": len(list(model.model.modules())),
        }

        # 모델 파일 크기 (best.pt)
        best_pt_path = os.path.join(results.save_dir, 'weights', 'best.pt')
        if os.path.exists(best_pt_path):
            model_summary["model/size_mb"] = os.path.getsize(best_pt_path) / (1024*1024)

        wandb.log(model_summary)

        # 3. 데이터셋 정보
        with open(yaml_path, 'r') as f:
            dataset_config = yaml.safe_load(f)

        dataset_info = {
            "dataset/classes": dataset_config['nc'],
            "dataset/names": str(dataset_config['names'])
        }
        wandb.log(dataset_info)

        # 4. 최종 성능 요약
        if hasattr(results, 'results_dict'):
            final_metrics = {
                "final/best_mAP50": results.results_dict.get('metrics/mAP50(B)', 0),
                "final/best_mAP50-95": results.results_dict.get('metrics/mAP50-95(B)', 0),
                "final/best_precision": results.results_dict.get('metrics/precision(B)', 0),
                "final/best_recall": results.results_dict.get('metrics/recall(B)', 0)
            }
            wandb.log(final_metrics)

        print("포괄적인 메트릭이 WandB에 로그되었습니다.")

    except Exception as e:
        print(f"포괄적 메트릭 로깅 중 오류: {e}")

In [14]:
def train_yolo_model(yaml_path=yaml_path, data_set_name=None,
                    model_name="yolov8m", epochs=30, batch_size=16,
                    wandb_project="codeit-ai-04-01", use_wandb=True):
    """
    향상된 WandB 통합이 포함된 YOLO 모델 훈련
    """
    model_save_name = f"{model_name}_{data_set_name}_{datetime.now(__kst).strftime('%Y%m%d_%H%M')}"

    print(f"model: {model_name}")
    print(f"yaml_path: {yaml_path}")
    print(f"data_set_name: {data_set_name}")
    print(f"model_save_name: {model_save_name}")
    print(f"use_wandb: {use_wandb}")

    # WandB 초기화
    if use_wandb:
        run = wandb.init(
            project=wandb_project,
            name=model_save_name,
            config={
                "model": model_name,
                "epochs": epochs,
                "batch_size": batch_size,
                "dataset": data_set_name,
                "imgsz": 640,
                "iou": 0.75,
                "optimizer": "AdamW",
                "lr0": 0.01
            },
            reinit=True
        )

    try:
        model = YOLO(f'{model_name}.pt')

        # 훈련 실행
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            batch=batch_size,
            iou=0.75,
            device=__device,
            project=get_path_modeling(),
            name=model_save_name,
            save_period=-1,
            patience=10,
            resume=False,
            verbose=False,
        )

        # 훈련 후 상세 메트릭 로깅
        if use_wandb and wandb.run is not None:
            log_comprehensive_metrics(results, model, yaml_path)

        return model, results, os.path.join(get_path_modeling(), model_save_name)

    except Exception as e:
        print(f"훈련 중 오류 발생: {e}")
        raise
    finally:
        if use_wandb and wandb.run is not None:
            wandb.finish()

In [15]:
print(unzip_paths)

['/content/code_it_ai04/yolo_noresize.zip.unzip']


In [16]:
os.environ['YOLO_VERBOSE'] = 'False'
os.environ['ULTRALYTICS_LOG_LEVEL'] = 'WARNING'  # 또는 'ERROR'

model_paths = []
#unzip_paths_sample = unzip_paths[:1]
unzip_paths_sample = unzip_paths
for path in unzip_paths_sample:
    key = extract_folder_key(path)
    yaml_path = os.path.join(path, "dataset.yaml")
    print("yaml_path:", yaml_path)
    print("path:", path)
    print("key:", key)
    update_yaml_paths_to_absolute(yaml_path)

    model, train_results, model_path= train_yolo_model(model_name="yolov8m", yaml_path=yaml_path, data_set_name=key)

    model_res={
        'key': key,
        'data_set_name' : key,
        'yolo_dataset_path': path,
        'yaml_path' : yaml_path,
        'model_path' : model_path,
    }
    model_paths.append(model_res)


yaml_path: /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
path: /content/code_it_ai04/yolo_noresize.zip.unzip
key: yolo_noresize
model: yolov8m
yaml_path: /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
data_set_name: yolo_noresize
model_save_name: yolov8m_yolo_noresize_20250918_1841
use_wandb: True


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.75, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_yolo_noresize_20250918_1841, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, p

dataset/classes,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
final/best_mAP50,▁
final/best_mAP50-95,▁
final/best_precision,▁
final/best_recall,▁
metrics/mAP50(B),▁▆▇█▇█████████████████████████
metrics/mAP50-95(B),▁▆▇▇▇▇▇███████████████████████
metrics/precision(B),▁▅▆▇▇▇▆▇▇▇▇▇▇█▇▇▇████▇██▇▇█▇██
metrics/recall(B),▁▅▆▇▇▇█▇██████████████████████
+9,...


In [17]:
for m in model_paths:
    print_json_tree(m)


|-- key(str): yolo_noresize
|-- data_set_name(str): yolo_noresize
|-- yolo_dataset_path(str): /content/code_it_ai04/yolo_noresize.zip.unzip
|-- yaml_path(str): /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
|-- model_path(str): /content/drive/MyDrive/modeling_yolo/yolov8m_yolo_noresize_20250918_1841


In [18]:

# 데이터 분포 확인
def print_dataset_info(yaml_path=yaml_path):
    with open(yaml_path, 'r') as f:
        dataset_config = yaml.safe_load(f)

    print('image:', os.path.join(dataset_config['path'], 'images'))
    train_images = [str(p) for p in Path(os.path.join(dataset_config['path'], dataset_config['train'])).glob("*.png")]
    val_images = [str(p) for p in Path(os.path.join(dataset_config['path'], dataset_config['val'])).glob("*.png")]
    test_images = [str(p) for p in Path(os.path.join(dataset_config['path'], dataset_config['test'])).glob("*.png")]

    train_images = [str(p) for p in Path(os.path.join(dataset_config['path'], dataset_config['train'])).glob("*.jpg")]
    val_images = [str(p) for p in Path(os.path.join(dataset_config['path'], dataset_config['val'])).glob("*.jpg")]
    test_images = [str(p) for p in Path(os.path.join(dataset_config['path'], dataset_config['test'])).glob("*.jpg")]

    print('label:', os.path.join(dataset_config['path'], 'labels'))
    train_labels = [str(p) for p in Path(os.path.join(dataset_config['path'], 'labels', 'train')).glob("*.txt")]
    val_labels = [str(p) for p in Path(os.path.join(dataset_config['path'], 'labels', 'val')).glob("*.txt")]
    test_labels = [str(p) for p in Path(os.path.join(dataset_config['path'], 'labels', 'test')).glob("*.txt")]

    return train_images, val_images,test_images, train_labels, val_labels, test_labels

# 제약코드를 카테고리 ID로 변환
def drug_to_category_id(drug_name):
    """약품명을 카테고리 ID로 변환"""
    if type(drug_name) is int:
        return drug_name
    if drug_name.startswith('K-'):
        return int(drug_name[2:]) - 1
    else:
        raise ValueError(f"Unexpected drug name format: {drug_name}")

def result_save_csv(csv_rows, save_csv_path):
    save_dir = os.path.dirname(save_csv_path)
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)
    with open(save_csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["annotation_id", "image_id", "category_id", "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"])
        writer.writerows(csv_rows)
    print(f"예측 결과가 {save_csv_path}에 저장되었습니다.")

def plot_result_view(results, class_names):
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()
    for idx, (result, ax) in enumerate(zip(results, axes)):
        img = result[0].orig_img
        annotated = result[0].plot()
        ax.imshow(annotated)
        ax.axis('off')
        if len(result[0].boxes) > 0:
            conf = result[0].boxes.conf[0].item()
            cls = int(result[0].boxes.cls[0].item())
            pred_class = class_names[cls] if cls < len(class_names) else 'unknown'
            ax.set_title(f'{pred_class} ({conf:.2f})')
        else:
            ax.set_title('No Detection')
    plt.tight_layout()
    plt.show()

def plot_result_save(results, class_names, save_dir="submission"):
    import os
    import cv2
    import random

    # 클래스별 색상 생성 (고정)
    random.seed(42)
    color_map = {}
    for i, cname in enumerate(class_names):
        # BGR 색상으로 직접 생성 (OpenCV는 BGR 사용)
        color_map[i] = tuple(random.choices(range(50, 256), k=3))

    save_dir = os.path.dirname(save_dir) if not os.path.isdir(save_dir) else save_dir
    if save_dir and not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)

    from tqdm import tqdm
    pbar = tqdm(enumerate(results), total=len(results), ascii=True, desc="Saving results")

    for idx, result in pbar:
        # 원본 이미지는 RGB 형식
        img_rgb = result[0].orig_img.copy()
        # OpenCV 작업을 위해 RGB를 BGR로 변환
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

        for box in result[0].boxes:
            xyxy = box.xyxy[0].cpu().numpy().astype(int)
            cls = int(box.cls.item())
            conf = box.conf.item()
            class_name = class_names[cls] if cls < len(class_names) else 'unknown'
            label = f"{class_name} {conf:.2f}"

            # BGR 색상 사용 (OpenCV는 BGR 순서)
            bgr_color = color_map.get(cls, (0, 255, 0))  # 기본값: 녹색 (B, G, R)

            # 박스 그리기
            cv2.rectangle(img_bgr, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), bgr_color, 2)

            # 라벨 위치 결정
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            label_y = xyxy[1] - 10
            if label_y < 0:
                label_y = xyxy[1] + label_size[1] + 5

            # 라벨 배경
            cv2.rectangle(
                img_bgr,
                (xyxy[0], label_y - label_size[1] - 5),
                (xyxy[0] + label_size[0], label_y + 5),
                bgr_color,
                -1
            )
            # 라벨 텍스트 (흰색으로 변경하여 가독성 향상)
            cv2.putText(img_bgr, label, (xyxy[0], label_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        img_path = result[-1]
        img_name = os.path.basename(img_path).split('.')[0]
        file_path = os.path.join(save_dir, f"result_{img_name}_yolo.jpg")

        # BGR 이미지를 직접 저장
        success = cv2.imwrite(file_path, img_bgr)
        if not success:
            print(f"Warning: Failed to save {file_path}")

        pbar.set_postfix_str(f"Saved: {os.path.basename(file_path)}")

In [19]:
for m in model_paths:
    print_json_tree(m)

    model_path = os.path.join(m['model_path'], "weights", "best.pt")
    yaml_path = os.path.join(m['yolo_dataset_path'], "dataset.yaml")
    print("model_path:", model_path)
    print("yaml_path:", yaml_path)

    # path 값을 절대경로로 변경한다.
    update_yaml_paths_to_absolute(yaml_path)

    with open(yaml_path, 'r') as f:
        dataset_config = yaml.safe_load(f)

    train_images, val_images, test_images, train_labels, val_labels, test_labels = print_dataset_info(yaml_path=yaml_path)
    class_names = dataset_config['names']
    # categories = {}
    # for cname in class_names:
    #     try:
    #         cid = drug_to_category_id(cname)
    #         categories[cname] = cid
    #     except ValueError as e:
    #         print(f"Warning: {e}")

    print("데이터셋 설정:")
    print(f"클래스 수: {dataset_config['nc']}")
    print(f"클래스 이름: {dataset_config['names']}")
    # print(f"카테고리: {categories}")
    print(f"훈련 경로: {dataset_config['train']}")
    print(f"검증 경로: {dataset_config['val']}")
    print(f"테스트 경로: {dataset_config['test']}")

    print(f"훈련 이미지: {len(train_images)}개 (라벨: {len(train_labels)}개)")
    print(f"검증 이미지: {len(val_images)}개 (라벨: {len(val_labels)}개)")
    print(f"테스트 이미지: {len(test_images)}개 (라벨: {len(test_labels)}개)")


|-- key(str): yolo_noresize
|-- data_set_name(str): yolo_noresize
|-- yolo_dataset_path(str): /content/code_it_ai04/yolo_noresize.zip.unzip
|-- yaml_path(str): /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
|-- model_path(str): /content/drive/MyDrive/modeling_yolo/yolov8m_yolo_noresize_20250918_1841
model_path: /content/drive/MyDrive/modeling_yolo/yolov8m_yolo_noresize_20250918_1841/weights/best.pt
yaml_path: /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
image: /content/code_it_ai04/yolo_noresize.zip.unzip/images
label: /content/code_it_ai04/yolo_noresize.zip.unzip/labels
데이터셋 설정:
클래스 수: 73
클래스 이름: [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732, 27776, 27925, 27992, 28762, 293

# TEST

In [20]:
#def test_on_samples(model, class_names, categories, test_files: list):
def test_on_samples(model, class_names, test_files: list):
    """테스트 이미지에서 샘플 예측 및 결과 CSV 저장"""
    if not test_files:
        print("테스트 이미지를 찾을 수 없습니다.")
        return

    print(f"{len(test_files)}개 테스트 이미지 예측 ===")

    pbar = tqdm(test_files, **get_tqdm_kwargs())
    results = []
    csv_rows = []
    json_rows = []
    annotation_id = 1

    for img_path in pbar:
        pbar.set_description(f"Processing {img_path}")
        res = model(img_path, verbose=False)
        results.append(res)

        # 이미지 파일명에서 숫자 추출 (예: '1.png' → 1)
        image_id = int(''.join(filter(str.isdigit, img_path)))

        # 각 박스별로 결과 저장
        for box in res[0].boxes:
            cls = int(box.cls.item())
            class_name = class_names[cls] if cls < len(class_names) else 'unknown'
            category_id = class_name
            # if class_name.startswith('K-'):
            #     category_id = categories[class_name]
            # else:
            #     raise ValueError(f"Unexpected class name format: {class_name}")

            score = box.conf.item()
            # bbox: [x_center, y_center, w, h] → [x, y, w, h]로 변환
            xywh = box.xywh[0].cpu().numpy()
            bbox_x = int(xywh[0] - xywh[2] / 2)
            bbox_y = int(xywh[1] - xywh[3] / 2)
            bbox_w = int(xywh[2])
            bbox_h = int(xywh[3])

            csv_rows.append([
                annotation_id, image_id, category_id, bbox_x, bbox_y, bbox_w, bbox_h, round(score, 2)
            ])
            json_rows.append({
                "annotation_id": annotation_id,
                "image_id": image_id,
                "img_path" : img_path,
                "category_id": category_id,
                "bbox": [bbox_x, bbox_y, bbox_w, bbox_h],
                "score": round(score, 3)
            })
            annotation_id += 1
    print(f"총 {len(csv_rows)}개의 예측 결과 생성됨.")
    return results, csv_rows, json_rows


In [21]:
def log_predictions_to_wandb(results, class_names, test_files, num_samples=20):
    """예측 결과를 WandB에 로그"""
    try:
        wandb_images = []
        sample_results = random.sample(list(zip(results, test_files)), min(num_samples, len(results)))

        for result, img_path in sample_results:
            # 원본 이미지
            img = Image.open(img_path)

            # 예측 결과가 있는 경우 바운딩 박스 정보 생성
            boxes = []
            if len(result[0].boxes) > 0:
                for box in result[0].boxes:
                    xyxy = box.xyxy[0].cpu().numpy()
                    cls = int(box.cls.item())
                    conf = box.conf.item()
                    class_name = class_names[cls] if cls < len(class_names) else 'unknown'

                    boxes.append({
                        "position": {
                            "minX": int(xyxy[0]),
                            "minY": int(xyxy[1]),
                            "maxX": int(xyxy[2]),
                            "maxY": int(xyxy[3])
                        },
                        "class_id": cls,
                        "box_caption": f"{class_name} ({conf:.2f})",
                        "domain": "pixel"
                    })

            wandb_images.append(
                wandb.Image(
                    img,
                    caption=f"Predictions: {os.path.basename(img_path)}",
                    boxes={"predictions": {"box_data": boxes, "class_labels": dict(enumerate(class_names))}}
                )
            )

        wandb.log({"predictions": wandb_images})

    except Exception as e:
        print(f"예측 결과 로깅 중 오류: {e}")

def create_wandb_summary_table(csv_rows, class_names):
    """예측 결과 요약 테이블 생성"""
    try:
        # 클래스별 예측 개수 계산
        class_counts = {}
        confidence_scores = []

        for row in csv_rows:
            category_id = row[2]
            score = row[7]

            # 카테고리 ID를 클래스명으로 변환
            class_name = f"K-{category_id + 1}"
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
            confidence_scores.append(score)

        # WandB 테이블 생성
        table = wandb.Table(columns=["Class", "Count", "Percentage"])
        total_predictions = len(csv_rows)

        for class_name, count in class_counts.items():
            percentage = (count / total_predictions) * 100 if total_predictions > 0 else 0
            table.add_data(class_name, count, f"{percentage:.1f}%")

        # 요약 메트릭
        avg_confidence = np.mean(confidence_scores) if confidence_scores else 0

        wandb.log({
            "summary/predictions_table": table,
            "summary/total_predictions": total_predictions,
            "summary/avg_confidence": avg_confidence,
            "summary/unique_classes": len(class_counts)
        })

    except Exception as e:
        print(f"요약 테이블 생성 중 오류: {e}")

In [22]:
def convert_yolo_bbox_with_orig_img(xywh, orig_img, default_orig_size=(976, 1280)):
    """
    YOLO 좌표를 원본 좌표로 변환

    Args:
        xywh: YOLO 바운딩 박스 좌표 [center_x, center_y, width, height]
        orig_img: YOLO에서 처리된 이미지 (리사이즈된 상태, 보통 640x640)
        default_orig_size: 실제 원본 이미지 크기 (width, height)

    Returns:
        tuple: (bbox_x, bbox_y, bbox_w, bbox_h) - 실제 원본 크기 기준 COCO 형식 좌표
    """

    orig_width, orig_height = default_orig_size
    yolo_height, yolo_width = orig_img.shape[:2]
    if yolo_height == orig_height and yolo_width == orig_width:
        # 리사이즈나 패딩이 없는 경우
        center_x, center_y, bbox_w, bbox_h = xywh
    else:
        # 리사이즈 및 패딩이 있는 경우 default_orig_size 기준 비율 유지 되었음으로 페딩을 제거하고 스케일 복원
        # default_orig_size -> orig_img 될 경우 패딩 계산 (비율 유지)
        # 페딩 제거 및 스케일 복원
        scale = min(yolo_width / orig_width, yolo_height / orig_height)
        resized_width = int(orig_width * scale)
        resized_height = int(orig_height * scale)
        pad_x = (yolo_width - resized_width) / 2
        pad_y = (yolo_height - resized_height) / 2

        center_x = (xywh[0] - pad_x) / scale
        center_y = (xywh[1] - pad_y) / scale
        bbox_w = xywh[2] / scale
        bbox_h = xywh[3] / scale

    # 중심점을 좌상단으로 변환
    bbox_x = center_x - bbox_w / 2
    bbox_y = center_y - bbox_h / 2

    # 경계 검사
    bbox_x = max(0, bbox_x)
    bbox_y = max(0, bbox_y)
    bbox_w = min(bbox_w, orig_width - bbox_x)
    bbox_h = min(bbox_h, orig_height - bbox_y)

    bbox_w = int(min(bbox_w, orig_width - bbox_x))
    bbox_h = int(min(bbox_h, orig_height - bbox_y))

    # 추가 검증
    if bbox_w <= 0 or bbox_h <= 0:
        print(f"Warning: Invalid bbox size - w:{bbox_w}, h:{bbox_h}")
        bbox_w = max(1, bbox_w)
        bbox_h = max(1, bbox_h)

    return int(bbox_x), int(bbox_y), int(bbox_w), int(bbox_h)

# 기존 함수도 유지 (호환성)
def convert_yolo_bbox_simple(xywh, orig_size=(976, 1280)):
    """기존 함수 - 호환성 유지"""
    return convert_yolo_bbox_with_orig_img(xywh, None, orig_size)

def test_on_samples_with_wandb(model, class_names, test_files: list, log_to_wandb=True):
    """테스트 이미지에서 샘플 예측 및 WandB 로깅"""
    if not test_files:
        print("테스트 이미지를 찾을 수 없습니다.")
        return

    print(f"{len(test_files)}개 테스트 이미지 예측 ===")

    pbar = tqdm(test_files, **get_tqdm_kwargs())
    results = []
    csv_rows = []
    json_rows = []
    annotation_id = 1

    for img_path in pbar:
        pbar.set_description(f"Processing {os.path.basename(img_path)}")
        res = model(img_path, verbose=False)
        res.append(img_path)  # 이미지 경로를 배열 마지막에 추가
        results.append(res)
        image_id = int(''.join(filter(str.isdigit, os.path.basename(img_path))))
        for box in res[0].boxes:
            cls = int(box.cls.item())
            class_name = class_names[cls] if cls < len(class_names) else 'unknown'
            category_id = class_name
            # if class_name.startswith('K-'):
            #     category_id = categories[class_name]
            # else:
            #     raise ValueError(f"Unexpected class name format: {class_name}")
            score = box.conf.item()
            xywh = box.xywh[0].cpu().numpy()

            bbox_x, bbox_y, bbox_w, bbox_h = convert_yolo_bbox_with_orig_img(
                xywh,
                res[0].orig_img,  # 실제 원본 이미지 전달
                default_orig_size=(976, 1280)  # 기본값
            )

            csv_rows.append([
                annotation_id, image_id, category_id, bbox_x, bbox_y, bbox_w, bbox_h, score
            ])
            json_rows.append({
                "annotation_id": annotation_id,
                "image_id": image_id,
                "img_path": img_path,
                "category_id": category_id,
                "bbox": [bbox_x, bbox_y, bbox_w, bbox_h],
                "score": score
            })
            annotation_id += 1

    print(f"총 {len(csv_rows)}개의 예측 결과 생성됨.")

    # WandB 로깅
    if log_to_wandb and wandb.run is not None:
        log_predictions_to_wandb(results, class_names, test_files)
        create_wandb_summary_table(csv_rows, class_names)

    return results, csv_rows, json_rows

In [23]:


for m in model_paths:
    print_json_tree(m)

    model_path = os.path.join(m['model_path'], "weights", "best.pt")
    yaml_path = os.path.join(m['yolo_dataset_path'], "dataset.yaml")
    now_str = datetime.now(__kst).strftime("%Y%m%d_%H%M%S")
    submission_path = os.path.join(m['model_path'], f"submission_{now_str}")

    print("model_path:", model_path)
    print("yaml_path:", yaml_path)

    # path 값을 절대경로로 변경한다.
    update_yaml_paths_to_absolute(yaml_path)

    with open(yaml_path, 'r') as f:
        dataset_config = yaml.safe_load(f)

    train_images, val_images, test_images, train_labels, val_labels, test_labels = print_dataset_info(yaml_path=yaml_path)
    class_names = dataset_config['names']

    #test_images = test_images[:3]

    print("데이터셋 설정:")
    print(f"클래스 수: {dataset_config['nc']}")
    print(f"클래스 이름: {dataset_config['names']}")
    print(f"훈련 경로: {dataset_config['train']}")
    print(f"검증 경로: {dataset_config['val']}")
    print(f"테스트 경로: {dataset_config['test']}")

    print(f"훈련 이미지: {len(train_images)}개 (라벨: {len(train_labels)}개)")
    print(f"검증 이미지: {len(val_images)}개 (라벨: {len(val_labels)}개)")
    print(f"테스트 이미지: {len(test_images)}개 (라벨: {len(test_labels)}개)")

    print("\n모델학습")
    model = YOLO(model_path)
    # results, csv_rows, json_rows = test_on_samples(model=model,
    #                 class_names=class_names,
    #                 categories=categories,
    #                 test_files=test_images)
    results, csv_rows, json_rows = test_on_samples_with_wandb(model=model,
                    class_names=class_names,
                    # categories=categories,
                    test_files=test_images)

    print('result_save_csv', result_save_csv)
    print('csv_rows', csv_rows)
    print('json_rows', json_rows)
    os.makedirs(submission_path, exist_ok=True)
    print(f"결과 저장 폴더: {submission_path}")

    csv_save_path = os.path.join(submission_path, f"submission.csv")
    result_save_csv(csv_rows, csv_save_path)
    plot_result_save(results, class_names, save_dir=submission_path)


|-- key(str): yolo_noresize
|-- data_set_name(str): yolo_noresize
|-- yolo_dataset_path(str): /content/code_it_ai04/yolo_noresize.zip.unzip
|-- yaml_path(str): /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
|-- model_path(str): /content/drive/MyDrive/modeling_yolo/yolov8m_yolo_noresize_20250918_1841
model_path: /content/drive/MyDrive/modeling_yolo/yolov8m_yolo_noresize_20250918_1841/weights/best.pt
yaml_path: /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml
image: /content/code_it_ai04/yolo_noresize.zip.unzip/images
label: /content/code_it_ai04/yolo_noresize.zip.unzip/labels
데이터셋 설정:
클래스 수: 73
클래스 이름: [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732, 27776, 27925, 27992, 28762, 293

Saving results: 100%|##########| 843/843 [00:14<00:00, 56.73it/s, Saved: result_807_yolo.jpg]
